In [4]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime

In [5]:
df_nov=pd.read_excel(r"..\Data\LineC_SessionData-individual rats_SampleData.xlsx")
df_march=pd.read_excel(r"..\Data\Snifftime data_March15.2019_example_withIndividualSniffs.xlsx")
df_sept=pd.read_excel(r"..\Data\AutomatedCage_Sept2023-24_raw data.xlsx")

In [13]:
rename_dict = {
    "RAT_NAME": "Rat_Name",
    "ID_EVALUATION_SESSION": "Evaluation_Session_ID",
    "RUN": "Run",
    "HOLE": "Hole",
    "SniffTime": "Sniff_Time",
    "tblEVALUATION_SniffThreshold": "Threshold_Time",
    "ID_BL_DOTS": "Lab_Test"
}

In [12]:
df_sept=df = df_sept.rename(columns=rename_dict)

NameError: name 'rename_dict' is not defined

In [23]:
def _positive_sample_rat(row): 
    if row['Sniff_Time']>=row['Threshold_Time']: 
        return 1 
    else: 
        return 0

def _positive_sample_lab(row): 
    if row['Lab_Test']==0:
        return pd.NA
    elif row['Lab_Test']>1: 
        return 1
    else: 
        return 0


def false_positive(rat, lab):
    """Return 1 if rat says positive but lab test is negative, else 0"""
    return 1 if rat == 1 and lab == 0 else 0

def false_negative(rat, lab):
    """Return 1 if rat says negative but lab test is positive, else 0"""
    return 1 if rat == 0 and lab == 1 else 0

def true_positive(rat, lab):
    """Return 1 if rat says positive and lab test is positive, else 0"""
    return 1 if rat == 1 and lab == 1 else 0

def true_negative(rat, lab):
    """Return 1 if rat says negative and lab test is negative, else 0"""
    return 1 if rat == 0 and lab == 0 else 0


In [24]:
df_sept['Rat_Positive_Sample']=df_sept.apply(lambda x: _positive_sample_rat(x),axis=1)
df_sept['Lab_Test']=df_sept.apply(lambda x: _positive_sample_lab(x),axis=1)
df["False_Positive"] = df.apply(lambda row: false_positive(row["Rat_Positive_Sample"], row["Lab_Test"]), axis=1)
df["False_Negative"] = df.apply(lambda row: false_negative(row["Rat_Positive_Sample"], row["Lab_Test"]), axis=1)
df["True_Positive"]  = df.apply(lambda row: true_positive(row["Rat_Positive_Sample"], row["Lab_Test"]), axis=1)
df["True_Negative"]  = df.apply(lambda row: true_negative(row["Rat_Positive_Sample"], row["Lab_Test"]), axis=1)


In [ ]:
# Select and rename columns, add empty columns, and calculate new columns for df_march
df_march_mod = df_march.rename(columns={
    'RAT_NAME': 'Rat_Name',
    'ID_EVALUATION_SESSION': 'Evaluation_Session_ID',
    'age': 'Age',
    'RUN': 'Run',
    'HOLE': 'Hole',
    'Sniff 1': 'Sniff_Time',
    'tblRAT_SESSION.SniffThreshold': 'Threshold_Time'
})

# Keep only the required columns (including those to be created)
columns_needed = [
    'Rat_Name',
    'Evaluation_Session_ID',
    'Gender',
    'Age',
    'Weight',
    'Run',
    'Hole',
    'Sniff_Time',
    'Threshold_Time',
    'Result_Of_Rat',
    'Lab_Test'
]

# Add empty columns for Gender and Weight
df_march_mod['Gender'] = ''
df_march_mod['Weight'] = ''
df_march_mod['Sniff_Time'] = df_march_mod['Sniff_Time'].fillna(0)

# Calculate Result_Of_Rat: 1 if Sniff_Time > Threshold_Time, else 0
df_march_mod['Result_Of_Rat'] = (df_march_mod['Sniff_Time'] > df_march_mod['Threshold_Time']).astype(int)

# Calculate Lab_Test: 1 if ID_BL_DOTS > 1, else 0 if == 1
df_march_mod['Lab_Test'] = df_march['ID_BL_DOTS'].apply(lambda x: 1 if x > 1 else (0 if x == 1 else None))

# Reorder columns to match the requested order
df_march_mod = df_march_mod[columns_needed]

# Display the first few rows to check
df_march_mod.head()

In [ ]:
# Create FP, FN, TP, TN columns based on Lab_Test (ground truth) and Result_Of_Rat (prediction)
df_march_mod['TP'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['TN'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)
df_march_mod['FP'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['FN'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)

df_march_mod.head()